<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/Mapa_de_Riesgos_Quito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análisis y modelado de vulnerabilidad ante riesgos naturales en Quito mediante Python, SIG y modelado espacial predictivo**

## Contenido
* [1. Introducción](#1-Introducción)
* [2. Importación de Librerías](#2-Importación-de-Librerías)
* [3. Carga y Descripción Inicial de Datos](#3-Carga-y-Descripción-Inicial-de-Datos)
* [4. Preprocesamiento y Transformación de Datos (ETL)](#4-Preprocesamiento-y-Transformación-de-Datos-(ETL)
  * [4.1 Conversión de TXT a CSV con Pandas](4.1-Conversión-de-TXT-a-CSV-con-Pandas)
  * [4.2 Limpieza de Datos](4.2-Limpieza-de-Datos)
  * [4.3 Ingeniería de Características (Feature Engineering)](4.3-Ingeniería-de-Características-(Feature-Engineering)
* [5. Integración de Datasets y Análisis Exploratorio Espacial](#5-Integración-de-Datasets-y-Análisis-Exploratorio-Espacial)
* [6. Modelado Predictivo de Riesgo](#6-Modelado-Predictivo-de-Riesgo)
* [7. Visualización Detallada de Resultados](#7-Visualización-Detallada-de-Resultados)
* [8. Conclusiones y Recomendaciones](#8-Conclusiones-y-Recomendaciones)
* [9. Referencias](#9-Referencias)

## 1. **Introducción**

La ciudad de Quito, capital de Ecuador, se encuentra en una zona geográfica compleja caracterizada por su relieve montañoso, su ubicación en el Cinturón de Fuego del Pacífico y su diversidad climática. Estas condiciones naturales hacen que la urbe esté expuesta a diferentes amenazas ambientales que ponen en riesgo a su población, infraestructura y ecosistemas urbanos.

Entre las amenazas más relevantes destacan:

- **Sismos:** Quito se ubica sobre una zona sísmicamente activa debido a la interacción de las placas tectónicas de Nazca y Sudamericana. Los movimientos telúricos representan uno de los mayores riesgos para la ciudad, especialmente en áreas con suelos blandos o edificaciones vulnerables.
- **Inundaciones:** Las precipitaciones intensas, combinadas con la expansión urbana y la deforestación de las laderas, incrementan la probabilidad de desbordamientos de quebradas y anegamientos en sectores bajos.
- **Olas de calor:** El cambio climático ha provocado un aumento de las temperaturas y una mayor frecuencia de eventos extremos, afectando la salud pública, la calidad del aire y el confort térmico en zonas densamente pobladas.

Frente a este contexto, resulta fundamental identificar las zonas más vulnerables de Quito mediante el uso de herramientas tecnológicas que integren información geográfica, estadística y ambiental. La programación en **Python** y los **Sistemas de Información Geográfica (SIG)** permiten analizar grandes volúmenes de datos espaciales y construir modelos predictivos que ayuden a la toma de decisiones basadas en evidencia.

Este proyecto busca aplicar técnicas de **análisis espacial y modelado predictivo** para generar un **mapa de vulnerabilidad ante riesgos naturales en Quito**, integrando diversas fuentes de datos abiertos.


### **Objetivo general:**
Analizar e identificar las zonas más vulnerables de Quito frente a amenazas naturales como sismos, inundaciones y olas de calor, utilizando herramientas de programación en Python, sistemas de información geográfica (SIG) y técnicas de modelado espacial para generar un mapa predictivo de riesgo y una visualización interactiva que facilite la interpretación de resultados.

### **Objetivos específicos:**
1. **Explorar y limpiar datasets abiertos** sobre peligros naturales (sísmicos, hidrológicos y climáticos) y características geográficas del Distrito Metropolitano de Quito.

2. **Integrar y georreferenciar capas de información** (geológica, hidrológica, topográfica, demográfica y climática) en un entorno de análisis geoespacial utilizando Python y librerías como `geopandas` y `rasterio`.

3. **Calcular indicadores de riesgo** combinando variables de amenaza, exposición y vulnerabilidad para cada zona o parroquia de Quito.

4. **Aplicar un modelo espacial predictivo** (por ejemplo, Random Forest espacial o Regresión logística geográfica) para estimar la probabilidad de riesgo en función de las características del terreno y los eventos históricos.

5. **Diseñar una visualización interactiva** de los resultados en Google Colab o mediante mapas dinámicos con `folium`o `plotly`, facilitando la interpretación de las áreas de mayor vulnerabilidad.

## 2. **Importación de Librerías**

In [3]:
# Importación de librerías principales
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")

## 3. **Carga y Descripción Inicial de Datos**

Los archivos que tienen datos sobre los sismos se encuentran en formato .txt lo que no es adecuado para el analisis. Se debe convertir a CSV

In [10]:
# Magnitud de los Sismos
archive_txt_mag = pd.read_csv(r'/content/cat_magnitud_2012-jul2025.txt',
                              sep='\t',
                              skiprows=26,
                              encoding='utf-8')
display(archive_txt_mag.head())

# Origen de los Sismos
archive_txt_origen = pd.read_csv(r'/content/cat_origen_2012-jul2025.txt',
                                 sep='\t',
                                 skiprows=52,
                                 encoding='utf-8')
display(archive_txt_origen.head())

# Tiempos de Onda de los Sismos
archive_txt_picks = pd.read_csv(r'/content/cat_picks_2012-jul2025.txt',
                                sep='\t',
                                skiprows=32,
                                encoding='utf-8')
display(archive_txt_picks.head())


,"#event,orig_id, magnitude_value, magnitude_uncertainty, magnitude_type, methodID, stationCount, agencyID"
0,"igepn2012bafq, 2766, 3.6, 0.33, MLv, ..."
1,"igepn2012bafq, 2766, 3.6, NaN, M, ..."
2,"igepn2012bbjo, 2772, 4.8, 0.23, MLv, ..."
3,"igepn2012bbjo, 2772, 4.8, NaN, M, ..."
4,"igepn2012bhrl, 2832, 5.5, 0.18, MLv, ..."


,"event, orig_id, time_value, time_value_ms, time_uncertainty, latitude_value, latitude_uncertainty, longitude_value, longitude_uncertainty, depth_value,depth_uncertainty, magnitude_value_M, magnitude_value_P,magnitude_type_P, magnitudeP_uncertainty, magnitudeP_stationCount, quality_associatedPhaseCount,quality_usedPhaseCount,quality_associatedStationCount,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente, methodID , earthModelID"
0,"igepn2012acvi, 2610, 2012-01-02 13:02:44.0..."
1,"igepn2012acxo, 2611, 2012-01-02 14:10:00.0..."
2,"igepn2012ahkn, 2643, 2012-01-05 01:18:17.0..."
3,"igepn2012ahzg, 2647, 2012-01-05 08:43:48.0..."
4,"igepn2012aigm, 2649, 2012-01-05 12:22:12.0..."


,"#event,orig_id,time_value,time_value_ms,network_code,station_code, channel_code, phase_code, arri_azimuth, arri_distance, arri_timeResidual,arri_weight, polarity, time_used, agencyID"
0,"igepn2012acvi, 2610, 2012-01-02 13:03:15.0..."
1,"igepn2012acvi, 2610, 2012-01-02 13:03:21.0..."
2,"igepn2012acvi, 2610, 2012-01-02 13:03:23.0..."
3,"igepn2012acvi, 2610, 2012-01-02 13:03:24.0..."
4,"igepn2012acvi, 2610, 2012-01-02 13:03:25.0..."


## 4. **Preprocesamiento y Transformación de Datos (ETL)**

In [ ]:
# Los datos de magnitud comienzan en la línea 28, por lo tanto, saltamos 27.
lineas_a_saltar_mag = 27

# Nombres de Columna para el archivo de Magnitud (8 columnas)
columnas_mag = [
    'event', 'orig_id', 'magnitude', 'magnitude_uncertainty',
    'magnitude_type', 'methodID', 'stationCount', 'agencyID'
]
separador_de_campos = r'\s*,\s*|,\s*|\s+'

## 4.1 **Conversión de TXT a CSV con Pandas**

## 4.2 **Limpieza de Datos**

## 4.3 **Ingeniería de Características (Feature Engineering)**

## 5. **Integración de Datasets y Análisis Exploratorio Espacial**

## 6. **Modelado Predictivo de Riesgo**

## 7. **Visualización Detallada de Resultados**

## 8. **Conclusiones y Recomendaciones**

## 9. **Referencias**